# How to Query an Open API the Right Way (without getting IP banned)

How many times have you wanted to collect historical data but failed due to goofing things up with the API? Were you trying to scrape a site and got your IP blocked for making too many requests? Or did something happen you're not really sure about? Today I'm going to break down the right way to collect data from an API.

### Collecting Historical Data

Today we're going to be collecting data from a crypto exchange's API. I'm trying to build a dataset of historical trades on a particular crypto futures contract, but that's not really what's important. *I'm trying to build a large dataset from an API that I only have free access to*. That implies limitations...  

### Considerations

If it's a free API, that's very nice of them. But they're not THAT nice. You can't send a query for terabytes of data and expect them to give it to you no problem. They will, however, give you that data if you spread out your ask. If you keep your requests within the boundaries of their willingness to be generous, you can accumulate a decently sized dataset over time. Therein lies the tradeoff though -- you spend more time in exchange for free access.  
In order to not get banned, we need to keep an eye on our rate limits. In other words, we need to make sure we aren't biting off more than they're willing to let us chew. We can be slow eaters and finish up the whole plate in due time for free, or in most cases we can pay up to scarf the whole thing down. 

###### Let's get started...

In [1]:
!pip freeze # these are just the packages I have installed for this task

certifi==2020.6.20
chardet==3.0.4
datatable==0.11.0
idna==2.10
numpy==1.19.4
pandas==1.1.4
python-dateutil==2.8.1
pytz==2020.4
requests==2.24.0
six==1.15.0
urllib3==1.25.11


For this tutorial, I'm going to be leveraging a few packages -- `json`, `requests`, `time`, and `datatable`. I'll explain why we need each of those as we go.

The `requests` package contains functions that allow us to make HTTP requests to get data from a URL. The way this works is you make a "GET" request to *get* data from the API. That returns a "response" object that has a number of different attributes we want to leverage. 

In [2]:
import requests
response = requests.get("https://www.bitmex.com/api/v1/trade?symbol=XBT%3Aperpetual&count=3&startTime=2020-11-03&endTime=2020-11-04")

Ok so let's break down that get request a bit:  
I'm trying to get data from this site: "https://www.bitmex.com/"  
Within their site structure, the added path leads to their API that has all the endpoints that will return data, making the url this: "https://www.bitmex.com/api/v1/". This is generally called the "Base" url.  
Within their API, there are several different [endpoints](https://www.bitmex.com/api/explorer/#!/) but the one we want is the trade data, which further extends the URL to specify the "trade" endpoint: "https://www.bitmex.com/api/v1/trade"  
Now, we need to get more specific with the trades we want. This is where all these extra arguments come in. You add the "?" to denote args are following, and specify them accordingly:  
- symbol="XBT perpetual"
- count=10
- startTime=2020-11-03
- endTime=2020-11-04  

Once we make this request using the `requests.get` command, we get a `response` object returned. **IF YOU DON'T WANT YOUR IP TO GET BANNED, PAY ATTENTION TO THE ATTRIBUTES OF THIS OBJECT**

In [3]:
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

The important ones to note are the ones that do **NOT** have underscores around them. When you make a request to an API, you're programmatically asking them for information. ***That can go a number of different ways***, but the way to check how it went is by printing the `response.status_code`

In [4]:
response.status_code

200

Ok great, a status code of 200 means it was OK. If you get a different one and you don't know what it means, just print `response.reason` to get an idea why your "programmatic ask" didn't work. Ofc google it too if you want more information on what could have gone wrong. But it's going to be imperative to check this, because it will give us insight on what we need to do in order to *keep collecting data* but keep the hand that feeds us happy.

### Who Gives a Fuck, Show Me the CONTENT

Right ok I get it you're here for the data, the last thing on your mind is making sure your IP doesn't get banned because you barely have any clue if you're querying things right. Ok, referencing that list of attributes in our response object again, we have a few options. `response.content` will show us what we have:

In [5]:
response.content

b'[{"timestamp":"2020-11-03T00:00:02.662Z","symbol":"XBTUSD","side":"Buy","size":941,"price":13566,"tickDirection":"ZeroMinusTick","trdMatchID":"4ca550d5-27af-5bcd-9693-a7e2fb52565a","grossValue":6936111,"homeNotional":0.06936111,"foreignNotional":941},{"timestamp":"2020-11-03T00:00:02.747Z","symbol":"XBTUSD","side":"Buy","size":7388,"price":13566,"tickDirection":"ZeroMinusTick","trdMatchID":"02a7438e-6ec5-868b-3ef4-36ea997ddabe","grossValue":54456948,"homeNotional":0.54456948,"foreignNotional":7388},{"timestamp":"2020-11-03T00:00:02.747Z","symbol":"XBTUSD","side":"Buy","size":7759,"price":13566,"tickDirection":"ZeroMinusTick","trdMatchID":"e558886b-0f88-7106-1ab7-02717e79747d","grossValue":57191589,"homeNotional":0.57191589,"foreignNotional":7759}]'

Nice, but that's one ugly mess of data. We can however, tell that is in JSON format because of the {"key":"value"} structure. **Most APIs will respond with data in this format**. That's cool and all but we're here to analyze data and I never learned how to analyze data in JSON format in my programming classes, only tables. So naturally, we're gonna get this data in a table.  

**MAJOR KEY**: BitMex allows you to return data in CSV format by adding `&_format=csv` to the end of your URL!!!! This is HUGE because `datatable` allows you to read csv's from URLs

In [6]:
import datatable as dt
dat = dt.fread("https://www.bitmex.com/api/v1/trade?symbol=XBT%3Aperpetual&count=3&startTime=2020-11-03&endTime=2020-11-04&_format=csv")
dat

,timestamp,symbol,side,size,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2020-11-03T00:00:02.662Z,XBTUSD,Buy,941,13566,ZeroMinusTick,4ca550d5-27af-5bcd-9693-a7e2fb52565a,6936111,0.0693611,941
1,2020-11-03T00:00:02.747Z,XBTUSD,Buy,7388,13566,ZeroMinusTick,02a7438e-6ec5-868b-3ef4-36ea997ddabe,54456948,0.544569,7388
2,2020-11-03T00:00:02.747Z,XBTUSD,Buy,7759,13566,ZeroMinusTick,e558886b-0f88-7106-1ab7-02717e79747d,57191589,0.571916,7759


Ok so now that we have proven we can get data on a single request and get it into a table, things start to get tricky. We want to get way more data than this. Like way more. But we can only get so much at a time. Let's try to get 10k records from the API -- we'll do this by pumping up that `count` parameter in the URL from 3 to 10,000.

In [7]:
r = requests.get("https://www.bitmex.com/api/v1/trade?symbol=XBT%3Aperpetual&count=10000&startTime=2020-11-03&endTime=2020-11-04&_format=csv")

Let's see if that worked:

In [8]:
r.status_code

400

Uh oh. 400 means... what?

In [9]:
r.reason

'Bad Request'

OOhhh ok it was a bad request? Fucking how?

In [10]:
r.content

b'{"error":{"message":"Maximum result count is 1000. Please use the start & count params to paginate.","name":"HTTPError"}}'

... maximum count is 1000. Makes sense. Can't take too big of a bite for free. They also say to use the start and count params to paginate, meaning those are the ways to make sure you're getting successive chunks of data. See, this is where it gets tricky. We can do that by making a while loop and not falsifying the condition it's running on until we've gotten all our data, which would presumably be the first query we see that has less than 1000 (the max) records. 

This is not going to work though. Can you guess why? If you just are constantly requesting, the website is going to get pissed at you for annoying them. It's like a kid asking a parent if "we're there yet" every 5 minutes on an 8 hour road trip. Like shut the fuck up. "We'll get there, WHEN WE GET THERE!" (name that movie). Ok seriously though you can't make too many requests in a certain time period (usually a minute). Most API documentations will have [this](https://www.bitmex.com/app/restAPI#Limits) section in their docs somewhere. It's all about limiting. There were a few big important things to note here:
1. The rate limit for free users is 30 requests per minute  
They let you know where you are in your rate limit with the 3 following headers (found in `r.headers`):
```
"x-ratelimit-limit": 60
"x-ratelimit-remaining": 58
"x-ratelimit-reset": 1489791662
```
Which is nice as fuck and we need to use them. That way, we can know that when the remaining hits 0, we can send the "x-ratelimit-reset" value straight to our sleep function to make sure we ask EXACTLY when we're allowed to again, wasting NO time.  

The NEXT thing to worry about is making sure you're asking for different stuff every time. You don't wanna ask for the same data over and over again, you need to ask for the next chunk of data to build your dataset. How do we do this? There are some different parameters we're able to include in the trade call, and even some other enpoints to hit that give us information about how much we should expect to get.  

The "start" argument to the `trade` endpoint denotes which trade to start on within your timeframe.
If we say we want to start on Jan 1, 2020 and get data up to Nov 4, 2020 -- it's going to give us the first 1k of ALL OF THOSE trades. What we'd have to do then is use the "start" argument to say "ok you gave me the first 1000, give me the next 1000 but start at trade 2000 this time". My worry is this is going to balloon into a MASSIVE integer, so it might be better to do the following:

The `trade/bucketed` endpoint gives information about how many trades were placed in a given time frame. Use that endpoint to get information about how many trades were placed ***per minute***. That way you can use the startTime as 2020-01-01 00:00:00.001 and the endTime as 2020-01-01 00:01:00 for example and use the "trades" field to inform your "start" arg to the `trade` endpoint. There will likely be more than 1000 trades placed per minute during most minutes of the day, but this will keep the number we use for the "start" arg down. Let's see that in action:

In [11]:
r1 = requests.get("https://www.bitmex.com/api/v1/trade/bucketed?binSize=1m&partial=false&symbol=XBT%3Aperpetual&count=1&reverse=false&startTime=2020-01-01T00%3A00%3A00.001Z&endTime=2020-01-01T00:01:00")

In [12]:
r1.json()

[{'timestamp': '2020-01-01T00:01:00.000Z',
  'symbol': 'XBTUSD',
  'open': 7163,
  'high': 7163,
  'low': 7156,
  'close': 7156,
  'trades': 2418,
  'volume': 2237930,
  'vwap': 7159.7337,
  'lastSize': 5,
  'turnover': 31258718411,
  'homeNotional': 312.58718410999995,
  'foreignNotional': 2237930}]

Ok so this is great. we know that in the first minute of 2020, there were 24 trades. We can use that trades object to control the start param for the `trade` endpoint.

In [13]:
r1.json()[0]['trades']

2418

So I'm going to skip ahead to the end and try to comment well why I'm doing what I'm doing, but the high level intuition lies in the analogy: You're a hungry kid and momma made pie. She's willing to give you some but you didn't pay her for the pie, so how much she gives you is beholden to her. You need to learn her rules for giving poor ass kids like you free pie so you can maximize the amount of pie you get without pissing her off. You need to make sure you ask her for different slices of the pie, only ask her for your alotted amount of pie at the exact frequency she allows, and to not ask for too much too often -- otherwise you will lose your privilege for free pie and go hungry.

In [ ]:
# the final loop

import time    # need this for sleeping and comparing timestamps
import os      # need this for checking if a file exists
from pathlib import Path  # same as above
import sys

start_time = '2020-01-01 00:00:00'
end_time = '2020-11-04 23:59:00'
trade_url = 'https://www.bitmex.com/api/v1/trade'
symbol = 'XBT:perpetual'
fname = Path('XBT_2020-01-01_2020-11-04.csv')

# go until timestamp meets end_time
while progress != end_time:
    
    r = requests.get(f"{trade_url}?symbol={symbol}&count=10000&startTime={start_time}&endTime=2020-11-04")
    try:
        # check the headers to find out where we're at in our asking
        remaining = r.headers['x-ratelimit-remaining']
        print(f"Requests remaining: {remaining}")
    except:
        print(r.status_code)
        print(r.reason)
        sys.exit()
    
    
    if remaining == '0':
        # sleep till rate limit resets
        print("Exceeded rate limit. Going to wait NOW.")
        while time.time() > float(r.headers['x-ratelimit-reset']):   # note this is in unix timestamps
            time.sleep(0.25) # quarter second
            print("Waiting...")
        continue
        
    start_time = 
    print(f"New Start Time: {start_time}")
    
    temp = dt.Frame(r.json())
    
    # if the file exists, append it. If not, write outright
    if fname.is_file():
        temp.to_csv(fname, append=True)    # if append=True, headers are not included (nice)
    else:
        temp.to_csv(fname)
    

In [ ]:
start_time = '2020-01-01 00:00:29.869000'
trade_url = 'https://www.bitmex.com/api/v1/trade'
symbol = 'XBT:perpetual'
r = requests.get(f"{trade_url}?symbol={symbol}&count=1&startTime={start_time}&endTime=2020-11-04")

In [ ]:
r.json()

In [ ]:
time.time() - float(r.headers['x-ratelimit-reset'])

### MAYBE LEARN

The "start" argument to the trade endpoint denotes which trade to start on within your timeframe
If we say we want to start on Jan 1, 2020 and go until Nov 4, 2020 -- it's going to give us the first 1k of ALL OF THOSE trades. What we'd have to do then is use the "start" argument to say "ok you gave me the first 1000, give me the next 1000 but start at trade 2000 this time". My worry is this is going to balloon into a MASSIVE integer, so it might be better to do the following:

Use the `trade/bucketed` endpoint to get information about how many trades were placed per minute. That way you can use the startTime as 2020-01-01 00:00:00 and the endTime as 2020-01-01 00:01:00 and use the "trades" field to inform your "start" arg to the `trade` endpoint



In [ ]:
import datetime
help(datetime.datetime)
start_time

In [35]:
import datetime
start_time = '2020-01-01 00:00:00'
end_time = '2020-11-04 23:59:00'

start = datetime.datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
end = datetime.datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S")

In [ ]:
import datetime
delta = start + datetime.timedelta(minutes=1)
delta.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
end_time == "2020-11-04 23:59:00"

In [33]:
start_time = '2020-01-01 00:00:00'
end_time = '2020-01-01 00:00:00'

In [36]:
start = start + datetime.timedelta(milliseconds=1)
start

datetime.datetime(2020, 1, 1, 0, 0, 0, 1000)

In [37]:
start = start.strftime("%Y-%m-%d %H:%M:%S.%f")
start

'2020-01-01 00:00:00.001000'

In [ ]:
#num_trades = r1.json()[0]['trades']
num_trades = 2418
pages = num_trades // 1000

In [ ]:
for i in range(pages):
    print(i)

In [ ]:
r.status_code == 400

In [ ]:
ar = requests.get("https://www.bitmex.com/api/v1/trade?symbol=XBT%3Aperpetual&count=1000&startTime=2020-01-01T00:00:00&endTime=2020-01-01T00:01:00&_format=csv")

In [ ]:
ar.reason

In [ ]:
test = dt.Frame(ar.content)

In [ ]:
test = dt.fread("https://www.bitmex.com/api/v1/trade?symbol=XBT%3Aperpetual&count=1000&startTime=2020-01-01T00:00:00&endTime=2020-01-01T00:01:00&_format=csv")

In [ ]:
test

In [ ]:
ar.status_code

In [26]:
r = requests.head("https://www.bitmex.com/api/v1/trade?symbol=XBT%3Aperpetual&count=10000&startTime=2020-11-03&endTime=2020-11-04&_format=csv")

In [27]:
r.headers

{'Date': 'Mon, 09 Nov 2020 19:51:43 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '121', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALBTG=4syji18ROVnyYW87Sm95YDcyQP26IyHwWpYYQo9qeu/TQkpxlFPLf+I2xG6KZlc9vGHolfCNI2xnxKKv3XkjHU+ruc2++FVeWHKFHEj/MUhlRvVPX45+zHSYTC35WTEbxSdWUqn7X0oCoS1kJYfJPW3RgWmTjl7kpR4FwlihDovt; Expires=Mon, 16 Nov 2020 19:51:43 GMT; Path=/, AWSALBTGCORS=4syji18ROVnyYW87Sm95YDcyQP26IyHwWpYYQo9qeu/TQkpxlFPLf+I2xG6KZlc9vGHolfCNI2xnxKKv3XkjHU+ruc2++FVeWHKFHEj/MUhlRvVPX45+zHSYTC35WTEbxSdWUqn7X0oCoS1kJYfJPW3RgWmTjl7kpR4FwlihDovt; Expires=Mon, 16 Nov 2020 19:51:43 GMT; Path=/; SameSite=None; Secure', 'x-ratelimit-limit': '30', 'x-ratelimit-remaining': '29', 'x-ratelimit-reset': '1604951504', 'X-Powered-By': 'Profit', 'Etag': 'W/"79-D7kVCDY55rXBYFyf/xbzGHQFn5E"', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains'}

In [28]:
r.url

'https://www.bitmex.com/api/v1/trade?symbol=XBT%3Aperpetual&count=10000&startTime=2020-11-03&endTime=2020-11-04&_format=csv'

In [15]:
d = dt.Frame()

TypeError: Cannot create Frame from <class 'bytes'>

In [16]:
start_time

NameError: name 'start_time' is not defined

In [40]:
dt.fread("https://www.bitmex.com/api/v1/trade?symbol=XBT:perpetual&count=1000&start=1000&startTime=2020-01-01%2000:00:00.001&endTime=2020-01-01%2000:01:00.000000&_format=csv")

,timestamp,symbol,side,size,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2020-01-01T00:00:29.869Z,XBTUSD,Sell,33,7161.5,ZeroMinusTick,df09a11b-261d-14ec-9682-c923e1da9d42,460812,0.00460812,33
1,2020-01-01T00:00:29.869Z,XBTUSD,Sell,26,7161.5,ZeroMinusTick,2d1095b6-2e98-b0d8-d126-2e471c711a8c,363064,0.00363064,26
2,2020-01-01T00:00:29.869Z,XBTUSD,Sell,33,7161.5,ZeroMinusTick,690bd13e-e165-c1a2-725f-98749b1b27ae,460812,0.00460812,33
3,2020-01-01T00:00:29.869Z,XBTUSD,Sell,23,7161.5,ZeroMinusTick,a71cc49f-c5a5-a129-6eb3-29fde7534cfd,321172,0.00321172,23
4,2020-01-01T00:00:29.869Z,XBTUSD,Sell,28,7161.5,ZeroMinusTick,7893fa32-8bfc-6d17-f015-77085dc00676,390992,0.00390992,28
5,2020-01-01T00:00:29.869Z,XBTUSD,Sell,23,7161.5,ZeroMinusTick,16913ccd-b58f-2f30-698e-0da7485e83ef,321172,0.00321172,23
6,2020-01-01T00:00:29.869Z,XBTUSD,Sell,32,7161.5,ZeroMinusTick,9d4df64a-f548-02a6-e1df-1b615990b861,446848,0.00446848,32
7,2020-01-01T00:00:29.869Z,XBTUSD,Sell,28,7161.5,ZeroMinusTick,4daf3fcb-b5fd-f603-b5f6-8d723edc114e,390992,0.00390992,28
8,2020-01-01T00:00:29.869Z,XBTUSD,Sell,25,7161.5,ZeroMinusTick,220b97d5-1e03-f8b6-6f30-34048b7d8b60,349100,0.003491,25


In [44]:
temp = dt.fread("https://www.bitmex.com/api/v1/trade?symbol=XBT:perpetual&count=1000&start=0&startTime=2020-01-01%2000:00:00.001&endTime=2020-01-01%2000:01:00.000000&_format=csv")

In [45]:
temp

,timestamp,symbol,side,size,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2020-01-01T00:00:02.780Z,XBTUSD,Sell,12,7162.5,MinusTick,3293e1ae-e0ac-485b-6d6e-2e538b7f5c31,167544,0.00167544,12
1,2020-01-01T00:00:02.793Z,XBTUSD,Sell,47,7162.5,ZeroMinusTick,876b320d-72b9-02af-283d-43ef6f3f3335,656214,0.00656214,47
2,2020-01-01T00:00:02.798Z,XBTUSD,Sell,428,7162.5,ZeroMinusTick,ac57aaef-6981-fe5b-e43e-6022cb22454d,5975736,0.0597574,428
3,2020-01-01T00:00:03.125Z,XBTUSD,Buy,1,7163,PlusTick,6a7f915b-8de9-7c36-0ead-834c85287730,13961,0.00013961,1
4,2020-01-01T00:00:03.125Z,XBTUSD,Buy,1,7163,ZeroPlusTick,b7c1563e-cda0-79fa-4483-caee6ac9241e,13961,0.00013961,1
5,2020-01-01T00:00:03.125Z,XBTUSD,Buy,1,7163,ZeroPlusTick,84008a01-4e73-acd1-845c-6ac0eb20c001,13961,0.00013961,1
6,2020-01-01T00:00:03.125Z,XBTUSD,Buy,1,7163,ZeroPlusTick,696782b9-a91e-b83a-37e7-1dfe9b5304ab,13961,0.00013961,1
7,2020-01-01T00:00:03.125Z,XBTUSD,Buy,1,7163,ZeroPlusTick,774dc668-4f02-eb96-f767-7d51d2076711,13961,0.00013961,1
8,2020-01-01T00:00:03.125Z,XBTUSD,Buy,1,7163,ZeroPlusTick,6eb97b31-c310-63d9-dec1-84f5fa4dc46d,13961,0.00013961,1


In [47]:
remaining = '0'
if remaining == '0':
    print("fuck")

fuck
